## Docker

```bash
sudo docker build -t maskrcnn-benchmark .
sudo docker run 
    --runtime=nvidia -d -it \
    --name=maskrcnn \
    -v=$(pwd):/notebooks \
    -p=8888:8888 \
    --ipc="host" \
    maskrcnn-benchmark
sudo docker logs maskrcnn
sudo docker start maskrcnn
sudo docker exec -it maskrcnn /bin/bash
```

## Notes

model-free vs model-based: model包含actor & critic，在critic方面，model-based有含renderer net, model-free沒有

In [9]:
# !pip install ray 
# ray[rllib] ray[debug] pandas

%cd /notebooks/LearningToPaint/rllib
!python canvas_env.py

/notebooks/LearningToPaint/rllib
W0828 14:48:39.492340 139928004024064 deprecation.py:323] From /miniconda/envs/py36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
2019-08-28 14:48:41,197	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-28_14-48-41_196248_1657/logs.
2019-08-28 14:48:41,340	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:24187 to respond...
2019-08-28 14:48:41,592	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:60879 to respond...
2019-08-28 14:48:41,597	INFO services.py:809 -- Starting Redis shard with 1.67 GB max memory.
2019-08-28 14:48:41,789	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-28_14-48-41_196248_1657/logs.


(pid=1697) 2019-08-28 14:49:09,686	INFO rollout_worker.py:742 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7f48965eaf60>}
(pid=1697) 2019-08-28 14:49:09,687	INFO rollout_worker.py:743 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.DictFlatteningPreprocessor object at 0x7f48965eacc0>}
(pid=1697) 2019-08-28 14:49:09,687	INFO rollout_worker.py:356 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f48965ea940>}
(pid=1697) 2019-08-28 14:49:09,822	INFO multi_gpu_optimizer.py:93 -- LocalMultiGPUOptimizer devices ['/cpu:0']
(pid=1697) Tensor("default_policy_1/tower/Reshape:0", shape=(?, 9, 9, 1), dtype=float32)
(pid=1697) Tensor("default_policy_1/tower_1/Reshape:0", shape=(?, 9, 9, 1), dtype=float32, device=/device:CPU:0)
(pid=1696) WARNING: Logging before flag parsing goes to stderr.
(pid=1696) W0828 14:49:22.532622 139833929369344 deprecation.py:323] From /miniconda/envs

(pid=1696)                    'obj_status': np.ndarray((1, 4), dtype=float32, min=0.0, max=3.0, mean=1.5),
(pid=1696)                    'target_im': np.ndarray((9, 9), dtype=uint8, min=0.0, max=255.0, mean=50.37)}}}
(pid=1696) 2019-08-28 14:49:52,884	INFO sampler.py:305 -- Info return from env: {0: {'agent0': None}}
(pid=1697) 2019-08-28 14:49:52,904	INFO trainer.py:366 -- Worker crashed during call to train(). To attempt to continue training without the failed worker, set `'ignore_worker_failures': True`.
2019-08-28 14:49:53,314	WARNING util.py:145 -- The `experiment_checkpoint` operation took 0.3674428462982178 seconds to complete, which may be a performance bottleneck.
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/0 GPUs
Memory usage on this node: 2.1/8.4 GB
Result logdir: /root/ray_results/PPO
Number of trials: 1 ({'ERROR': 1})
ERROR trials:
 - PPO_CanvasEnv_0:	ERROR, 1 failures: /root/ray_results/PPO/PPO_CanvasEnv_0_2019-08-28_14-48-42uh0ik3s8/err

In [8]:
# install openai baselines and other required packages
# !pip install gym tensorflow

# !apt-get update && apt-get -y install cmake libopenmpi-dev python3-dev zlib1g-dev
# %cd /notebooks/baselines
# # !git clone https://github.com/openai/baselines.git  # use github-desktop to clone repository
# !pip install -e .

# # install local gym-canvas as package
# %cd /notebooks/LearningToPaint/gym-canvas
# !pip install -e .

## Training

### train renderer

In [52]:
%cd /notebooks/LearningToPaint/pytorch-a2c-ppo-acktr-gail
!python main.py --env-name "gym_canvas:canvas-v0" --num-processes 1 --algo ppo --use-gae --lr 2.5e-4 --clip-param 0.1 --value-loss-coef 0.5 --num-steps 5 --num-mini-batch 4 --log-interval 1 --use-linear-lr-decay --entropy-coef 0.01

# !python baseline/train_renderer.py
# !tensorboard --logdir train_log --port=6006

/notebooks/LearningToPaint/pytorch-a2c-ppo-acktr-gail
Tuple
Traceback (most recent call last):
  File "main.py", line 254, in <module>
    main()
  File "main.py", line 55, in main
    base_kwargs={"recurrent": args.recurrent_policy},
  File "/notebooks/LearningToPaint/pytorch-a2c-ppo-acktr-gail/a2c_ppo_acktr/model.py", line 41, in __init__
    raise NotImplementedError
NotImplementedError


### train RL

* make data available

In [10]:
!python baseline/train.py --max_step=4 --env_batch=1 --warmup=1 --validate_interval=1 --debug

mkdir: cannot create directory './model': File exists
Renderer model loaded.
observation_space (1, 128, 128, 7) action_space 13
 #0: steps:4 interval_time:0.22 train_time:24.56
evaluating
 Step_0000007: mean_reward:0.288 mean_dist:0.201 var_dist:0.000
 #1: steps:8 interval_time:2.69 train_time:34.23
evaluating
 Step_0000011: mean_reward:-8.902 mean_dist:0.619 var_dist:0.000
 #2: steps:12 interval_time:2.55 train_time:43.24
evaluating
 Step_0000015: mean_reward:-7.753 mean_dist:0.417 var_dist:0.000
 #3: steps:16 interval_time:2.68 train_time:55.46
evaluating
 Step_0000019: mean_reward:-1.631 mean_dist:0.326 var_dist:0.000
^C


### Supervised training

In [1]:
!python baseline/train_supervised.py

Renderer model loaded.
step 0: loss 9167.975586
^C
Traceback (most recent call last):
  File "baseline/train_supervised.py", line 84, in <module>
    train(args.batch_size, args.episode_steps)